In [109]:
import math
import json

import pandas as pd

from tqdm import tqdm

import requests
import html5lib
from bs4 import BeautifulSoup

from stem import Signal
from stem.control import Controller

from fake_useragent import UserAgent

from tqdm.notebook import tqdm

In [2]:
def get_request(url):
    renew_tor_ip()
    
    user_agent = UserAgent().chrome
    headers = {'User-Agent': user_agent}
    
    
    session = requests.session()

    session.proxies = {}
    session.proxies['http']='socks5h://localhost:9050'
    session.proxies['https']='socks5h://localhost:9050'

    try:
        r = session.get(url, headers=headers)
    except Exception as e:
        print(str(e))
    else:
        return r   

def renew_tor_ip():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="tor_for_parsing")
        controller.signal(Signal.NEWNYM)

In [4]:
url_with_phones = 'https://www.e-katalog.ru/ek-list.php?katalog_=122&order_=reviews&page_={}'

pages_to_scrape = 28
phones_with_reviews = []

for page_num in range(pages_to_scrape):
    response = get_request(url_with_phones.format(page_num))
    soup = BeautifulSoup(response.text, 'html5lib')
    phones = soup.find_all('table', {'class': 'model-short-block'})
    
    for phone in phones:
        phone_first_param = phone.select_one('a[class="ib"]')
        if phone_first_param:
            reviews = phone_first_param.find('u')
            reviews_link = phone_first_param['link']
            if reviews.text == 'Отзывы' and '/review/' in reviews_link:
                phones_with_reviews.append(reviews_link) if reviews_link not in phones_with_reviews else None
            else:
                break
        else:
            break
    
    print(len(phones_with_reviews))

24
48
72
96
120
144
168
192
216
240
264
288
312
336
360
382
405
425
449
473
497
521
543
565
587
611
631
631


In [5]:
main_url = 'https://www.e-katalog.ru'

In [6]:
phones_with_reviews[0]

'/review/xiaomi-redmi-note-9-pro-128gb/'

In [105]:
def append_first_10_reviews_to_df(page_soup, data_frame):

    reviews = page_soup.find_all('div', {'class': 'review'})

    for review in reviews:

        review_title = review.find('div', {'class': 'review-title'})
        review_type = review_title.find('img').get('src')

        review_description_elements = []

        review_description_elements.append(review_title)
        review_description_elements.append(review.find('div', {'class': 'review-comment'}))
        review_description_elements.append(review.find('div', {'class': 'review-plus'}))
        review_description_elements.append(review.find('div', {'class': 'review-minus'}))

        review_description_elements_filled = []

        for review_element in review_description_elements:
            if review_element:
                review_description_elements_filled.append(review_element.text)

        review_text = ' '.join(review_description_elements_filled).replace('\n', ' ').replace('  ', ' ')
        
        data_frame = data_frame.append({'review': review_text, 'type': review_type}, ignore_index=True)

    return data_frame

In [110]:
df = pd.DataFrame(columns=['review', 'type'])

In [111]:
for phone in tqdm(phones_with_reviews):

    response = get_request(main_url + phone)
    soup = BeautifulSoup(response.text, 'html5lib')
    reviews = soup.find_all('div', {'class': 'review'})
    
    df = append_first_10_reviews_to_df(soup, df)

In [118]:
df = df.drop_duplicates()

In [119]:
df.to_csv('e-catalog_phone_reviews.csv')